In [181]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

from sklearn import model_selection
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot
import graphviz
from sklearn.tree import export_graphviz
import math

# 단톡방 과제 2
- 60171100 윤민혁
- 타이타닉 데이터를 의사결정나무 모형으로 적합시키고 변수중요도를 구하시오

In [182]:
# 데이터 불러오기
df = pd.read_csv('./data/titanic.csv', index_col=['PassengerId'])
# 데이터 전처리
df = df.drop({'Name','Ticket','Cabin','Embarked'}, axis = 1)
df.Age.fillna(df.Age.median(), inplace = True)
df['Sex'] = df.Sex.map({'female':0,'male':1})

#input변수, output변수
X = np.array(df.iloc[:,1:])
y = np.array(df['Survived'])

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)

tree = DecisionTreeClassifier(random_state=0, max_depth=3)
tree.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, random_state=0)

In [183]:
# 성능평가
temp_y_pred = tree.predict(X_test)
temp_acc = accuracy_score(y_test, temp_y_pred)

In [184]:
# 변수 중요도 확인
tree.feature_importances_

array([0.17568873, 0.61552198, 0.06794334, 0.08767138, 0.        ,
       0.05317456])

In [185]:
# 데이터 프레임으로 변환
pd.DataFrame(tree.feature_importances_.reshape(1,-1), columns= df.columns[1:], index = ['feature_importance'])

,Pclass,Sex,Age,SibSp,Parch,Fare
feature_importance,0.175689,0.615522,0.067943,0.087671,0.0,0.053175


## 원핫인코딩으로 탑승항구 변환

In [186]:
# 데이터 불러오기
df1 = pd.read_csv('./data/titanic.csv', index_col=['PassengerId'])
# 데이터 전처리
df1 = df1.drop({'Name','Ticket','Cabin'}, axis = 1)
df1.Age.fillna(df1.Age.median(), inplace = True)
df1 = df1.dropna()
df1['Sex'] = df1.Sex.map({'female':0,'male':1})

df1 = df1.reset_index(drop = True)
df1

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,S
1,1,1,0,38.0,1,0,71.2833,C
2,1,3,0,26.0,0,0,7.9250,S
3,1,1,0,35.0,1,0,53.1000,S
4,0,3,1,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
884,0,2,1,27.0,0,0,13.0000,S
885,1,1,0,19.0,0,0,30.0000,S
886,0,3,0,28.0,1,2,23.4500,S
887,1,1,1,26.0,0,0,30.0000,C


In [187]:
# 원한인코더를 encoder라는 변수에 저장
encoder = OneHotEncoder()
#encoder를 사용해 Embarked 항목 값으로 학습
encoder.fit(df1[['Embarked']])
#encoder의 transform을 사용해 embarked 변수를 변환해 onehot이라는 변수에 저장
onehot = encoder.transform(df1[['Embarked']])
# onehot이라는 변수를 array 형태로 변환
onehot = onehot.toarray()
# onehot이라는 변수를 DataFrame 형태로 변환
onehot = pd.DataFrame(onehot)
# 칼럼 이름 직점생성
onehot.columns = ['C','Q','S']
# 결과확인
onehot

,C,Q,S
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0
...,...,...,...
884,0.0,0.0,1.0
885,0.0,0.0,1.0
886,0.0,0.0,1.0
887,1.0,0.0,0.0


In [188]:
# 기존 df에 인코딩 결과를 병합
onehot = pd.concat([df1, onehot], axis = 1)

In [189]:
# 인코딩 전 'embarked'변수 제거 후 df에 업데이트
df_concat = onehot.drop('Embarked', axis =1)
df_concat

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,C,Q,S
0,0,3,1,22.0,1,0,7.2500,0.0,0.0,1.0
1,1,1,0,38.0,1,0,71.2833,1.0,0.0,0.0
2,1,3,0,26.0,0,0,7.9250,0.0,0.0,1.0
3,1,1,0,35.0,1,0,53.1000,0.0,0.0,1.0
4,0,3,1,35.0,0,0,8.0500,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
884,0,2,1,27.0,0,0,13.0000,0.0,0.0,1.0
885,1,1,0,19.0,0,0,30.0000,0.0,0.0,1.0
886,0,3,0,28.0,1,2,23.4500,0.0,0.0,1.0
887,1,1,1,26.0,0,0,30.0000,1.0,0.0,0.0


In [191]:
X = np.array(df_concat.iloc[:,1:])
y = np.array(df_concat['Survived'])

In [192]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)

tree_concat = DecisionTreeClassifier(random_state=0, max_depth=3)
tree_concat.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, random_state=0)

In [194]:
#성능 평가
temp_y_pred1 = tree_concat.predict(X_test)
temp_acc_concat = accuracy_score(y_test, temp_y_pred1)

In [195]:
a = tree_concat.feature_importances_
a

array([0.20212022, 0.59958245, 0.0691925 , 0.06662978, 0.        ,
       0.06247505, 0.        , 0.        , 0.        ])

In [197]:
# 데이터 프레임으로 변환
pd.DataFrame(a.reshape(1,-1), columns= df_concat.columns[1:], index = ['feature_importance'])

,Pclass,Sex,Age,SibSp,Parch,Fare,C,Q,S
feature_importance,0.20212,0.599582,0.069193,0.06663,0.0,0.062475,0.0,0.0,0.0


In [198]:
print('항구정보 미포함', format(temp_acc))
print('항구정보 포함', format(temp_acc_concat))

항구정보 미포함 0.8208955223880597
항구정보 포함 0.7902621722846442


- 변수중요도를 보면 탑승항구 변수의 주요도는 0이므로 탑승항구정보는 관련이 없다는 것을 알 수 있다.
- 따라서 정확도도 항구 정보를 포함하지 않는 것이 더 높게 나오는 것을 확인할 수 있다.